This notebook is associated with the paper "The relative class number one problem for function fields, I" by K.S. Kedlaya. It runs in SageMath (tested using version 9.5beta9).

In this notebook, we use the linear programming method (a/k/a Weil's explicit formulas) to compute some effective upper bounds on the number of rational points on a curve over a finite field. These bounds are ultimately derived from the positivity condition: for each positive integer $i$, the number of degree-$i$ places of the curve must be nonnegative.

In [1]:
load("../Shared/weil_poly_utils.sage")

In [2]:
def psi(t, l, n=1):
    P.<z> = LaurentPolynomialRing(RR)
    c = 1 + 2*sum(i^2 for i in l)
    f = 1/c*(1 + sum(l[i]*(z^(i+1)+z^(-i-1)) for i in range(len(l))))^2
    d = f.degree()
    s = f.coefficients()
    return sum(t^i*s[d+i] for i in range(1, d+1) if i%n==0)

Compute constants in a bound of the form
$$
    \#C(\mathbb{F}_q) \leq c_0 g + c_1
$$
where $C$ is a curve of genus $g$ over $\mathbb{F}_q$.

In [3]:
l = [1, .7, .2]
for q in [2,3,4]:
    print(1/psi(q^-0.5, l), 1 + psi(q^0.5, l)/psi(q^-0.5, l))

0.825950883211498 5.34527346707773
1.15251691701741 11.6683862113728
1.43462897526502 21.7491166077738


In [4]:
l = [1.05, 0.8, .4]
q = 8
print(1/psi(q^-0.5, l), 1 + psi(q^0.5, l)/psi(q^-0.5, l))

2.24313315986426 182.808168035954


Compute constants in a bound of the form
$$
    a_1 + c_2 (2a_2) + c_3 (3a_3) + c_4 (4a_4) \leq c_0 g + c_1
$$
where $a_i$ is the number of degree $i$-places on some curve of genus $g$ over $\mathbb{F}_2$. For convenience, in practice we will round $c_0$ and $c_1$ up and round $c_2,c_3,c_4$ down.

In [5]:
l = [1, 0.85, 0.25]
c = [1/psi(1/2^0.5, l), psi(2^0.5, l)/psi(1/2^0.5, l)+1]
print([ceil(i*10^4)/10^4.0 for i in c])
c = [psi(1/2^0.5, l, i)/psi(1/2^0.5, l) for i in range(1,5)]
print([floor(i*10^4)/10^4.0 for i in c])

[0.804200000000000, 5.61900000000000]
[1.00000000000000, 0.336600000000000, 0.138200000000000, 0.0537000000000000]


Table of Weil polynomials of simple abelian varieties of order 1 and dimension at most 6, taken from LMFDB.

In [6]:
P.<T> = QQ[]
simple_poly = {'1.2.ac': T^2 - 2*T + 2,
               '2.2.a_ae': T^4 - 4*T^2 + 4,
               '2.2.ad_f': T^4 - 3*T^3 + 5*T^2 - 6*T + 4,
               '2.2.ac_c': T^4 - 2*T^3 + 2*T^2 - 4*T + 4,
               '2.2.ab_ab': T^4 - T^3 - T^2 - 2*T + 4,
               '3.2.ad_c_b': T^6 - 3*T^5 + 2*T^4 + T^3 + 4*T^2 - 12*T + 8,
               '3.2.ae_j_ap': T^6 - 4*T^5 + 9*T^4 - 15*T^3 + 18*T^2 - 16*T + 8,
               '4.2.af_m_au_bd': T^8 - 5*T^7 + 12*T^6 - 20*T^5 + 29*T^4 - 40*T^3 + 48*T^2 - 40*T + 16,
               '4.2.ae_g_ae_c': T^8 - 4*T^7 + 6*T^6 - 4*T^5 + 2*T^4 - 8*T^3 + 24*T^2 - 32*T + 16,
               '4.2.ad_c_a_b': T^8 - 3*T^7 + 2*T^6 + T^4 + 8*T^2 - 24*T + 16,
               '4.2.ae_f_c_al': T^8 - 4*T^7 + 5*T^6 + 2*T^5 - 11*T^4 + 4*T^3 + 20*T^2 - 32*T + 16,
               '4.2.ae_e_h_av': T^8 - 4*T^7 + 4*T^6 + 7*T^5 - 21*T^4 + 14*T^3 + 16*T^2 - 32*T + 16,
               '4.2.af_n_az_bn': T^8 - 5*T^7 + 13*T^6 - 25*T^5 + 39*T^4 - 50*T^3 + 52*T^2 - 40*T + 16,
               '6.2.ag_p_av_y_abn_cn': T^12 - 6*T^11 + 15*T^10 - 21*T^9 + 24*T^8 - 39*T^7 + 65*T^6 - 78*T^5 + 96*T^4 - 168*T^3 + 240*T^2 - 192*T + 64,
               '6.2.af_j_ah_d_ab_ab': T^12 - 5*T^11 + 9*T^10 - 7*T^9 + 3*T^8 - T^7 - T^6 - 2*T^5 + 12*T^4 - 56*T^3 + 144*T^2 - 160*T + 64,
               '6.2.ag_p_at_g_bb_acj': T^12 - 6*T^11 + 15*T^10 - 19*T^9 + 6*T^8 + 27*T^7 - 61*T^6 + 54*T^5 + 24*T^4 - 152*T^3 + 240*T^2 - 192*T + 64
              }

Generate statistics about simple abelian varieties of order 1, including the $\LaTeX$ table.

In [7]:
def trace_from_weil_poly(u, n):
    Q.<t> = PowerSeriesRing(QQ)
    v = u.reverse()(t).log()
    l = v.list()
    while len(l) <= n+1:
        l.append(0)
    return [-l[i]*i for i in range(1, n+1)]

In [8]:
cc = [0.8042, 5.619, 1, 0.3366, 0.1137, 0.0537]
min_excess = 5
for u in simple_poly.values():
    d = u.degree() // 2
    tmp = trace_from_weil_poly(u, 4)
    excess = (1 + cc[3])*tmp[0] + cc[3]*tmp[1] + cc[4]*(tmp[2]-tmp[0]) + cc[5]*(tmp[3]-tmp[1])
    excess_scaled = excess / d
    excess_scaled = (excess_scaled*10^4).trunc() * 10.0^(-4)
    min_excess = min(min_excess, excess_scaled)
print(min_excess)

1.56120000000000


In [9]:
import re

print(r'\begin{tabular}{c||c||c|c|c|c||c|c|c|c}')
print(r'Label & $n$ & $T_{A,2}$ & $T_{A,4}$ & $T_{A,8}$ & $T_{A,16}$ & $T_{A,2} + T_{A,4}$ & $2T_{A,2} + T_{A,4}$ & $3 T_{A,2} + T_{A,4} $ & excess\\')
print(r'\hline')
d0 = 0
for label, u in simple_poly.items():
    tmp = u.factor()
    assert len(tmp) == 1
    F.<a> = NumberField(tmp[0][0])
    eta = (a^2-a)/(2-a)
    n = eta.multiplicative_order()
    assert (n < Infinity)
    tmp = trace_from_weil_poly(u, 4)
    excess = (1 + cc[3])*tmp[0] + cc[3]*tmp[1] + cc[4]*(tmp[2]-tmp[0]) + cc[5]*(tmp[3]-tmp[1])
    excess = (excess*10^4).round() * 10.0^(-4)
    d = u.degree() // 2
    if d0 < d:
        print(r'\hline')
        d0 = d
    excess = excess - d*min_excess
    ans = (n, tmp[0], tmp[1], tmp[2], tmp[3], tmp[0]+tmp[1], 2*tmp[0]+tmp[1], 3*tmp[0]+tmp[1])
    ans = ' & '.join('$' + str(i) + '$' for i in ans)
    ans = r'\avlink{' + re.sub(r'_', r'\_', label) + '} & ' + ans + ' & ' + f'{excess:.4f}'
    print(ans + r' \\')
print(r'\end{tabular}')

\begin{tabular}{c||c||c|c|c|c||c|c|c|c}
Label & $n$ & $T_{A,2}$ & $T_{A,4}$ & $T_{A,8}$ & $T_{A,16}$ & $T_{A,2} + T_{A,4}$ & $2T_{A,2} + T_{A,4}$ & $3 T_{A,2} + T_{A,4} $ & excess\\
\hline
\hline
\avlink{1.2.ac} & $2$ & $2$ & $0$ & $-4$ & $-8$ & $2$ & $4$ & $6$ & 0.0002 \\
\hline
\avlink{2.2.a\_ae} & $1$ & $0$ & $8$ & $0$ & $16$ & $8$ & $8$ & $8$ & 0.0000 \\
\avlink{2.2.ad\_f} & $3$ & $3$ & $-1$ & $0$ & $7$ & $2$ & $5$ & $8$ & 0.6393 \\
\avlink{2.2.ac\_c} & $4$ & $2$ & $0$ & $8$ & $8$ & $2$ & $4$ & $6$ & 0.6626 \\
\avlink{2.2.ab\_ab} & $6$ & $1$ & $3$ & $10$ & $-1$ & $4$ & $5$ & $6$ & 0.0325 \\
\hline
\avlink{3.2.ad\_c\_b} & $7$ & $3$ & $5$ & $6$ & $-11$ & $8$ & $11$ & $14$ & 0.4911 \\
\avlink{3.2.ae\_j\_ap} & $7$ & $4$ & $-2$ & $1$ & $10$ & $2$ & $6$ & $10$ & 0.2929 \\
\hline
\avlink{4.2.af\_m\_au\_bd} & $5$ & $5$ & $1$ & $5$ & $-3$ & $6$ & $11$ & $16$ & 0.5600 \\
\avlink{4.2.ae\_g\_ae\_c} & $8$ & $4$ & $4$ & $4$ & $0$ & $8$ & $12$ & $16$ & 0.2332 \\
\avlink{4.2.ad\_c\_a\_b} & $10$ & 

Compute upper bounds on $g$ and $g'$ for $q=2$ with no restriction on $d$.

In [10]:
tmp1 = 1 + 2*cc[3]
tmp = [1 + tmp1*cc[0] / min_excess, tmp1*cc[1] / min_excess]
tmp = [floor(i * 10^4) / 10^4.0 for i in tmp]
print(tmp)

[1.86180000000000, 6.02210000000000]


In [11]:
print([floor(tmp[0]*g + tmp[1]) for g in range(8)])

[6, 7, 9, 11, 13, 15, 17, 19]


Improve these bounds using better upper bounds on point counts.

In [12]:
point_count_bounds = [(3,5), (5,9), (6,10), (7,14), (8,15), (9, 17), \
                      (10, 20), (10, 21), (11, 23), (12, 26), (13, 27)]

In [13]:
print([g + floor((cc[0]*g + cc[1] + 0.6732*point_count_bounds[g][0]) / min_excess) for g in range(8)])

[4, 7, 9, 11, 13, 15, 17, 18]


Compute upper bounds on $g$ and $g'$ for $q=2$ when $d=2$.

In [14]:
top = floor((min_excess + cc[1]) / (min_excess - cc[0]))
print(top)
print([floor((cc[0]*g + cc[1] - min_excess*(g-1))/(min_excess - cc[3])) + 2*g-1 \
             for g in range(1, top+1)])

9
[6, 7, 9, 10, 11, 13, 14, 15, 17]
